### Load Libraries

In [1]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}

Intitializing Scala interpreter ...

Spark Web UI available at http://b2cc1a63ffdb:4042
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559653926775)
SparkSession available as 'spark'


2019-06-04 13:12:03,059 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-06-04 13:12:06,276 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2019-06-04 13:12:06,283 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}


### Data loading

In [2]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [65]:
// combine the 3 dataframes to 1
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [4]:
// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

2019-06-04 13:13:14,573 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


### Data cleansing and reshaping

In [66]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [67]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [69]:
//string array of features

var creditRiskFeatures = creditRiskdf.columns
creditRiskFeatures = creditRiskFeatures.filter(! _.contains("TARGET"))

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [70]:
creditRiskFeatures.length

res35: Int = 281


In [71]:
creditRiskdf.cache()

2019-06-04 22:07:27,745 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.


res36: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [9]:
// subset of features for testing of outliers code

// val c1 = creditRiskFeatures.slice(20,40)

c1: Array[String] = Array(previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CREDIT_mean, client_installments_AMT_PAYMENT_sum_mean, client_installments_DAYS_INSTALMENT_min_max, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_AMT_INSTALMENT_max_max, client_cash_CNT_INSTALMENT_FUTURE_min_mean, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, client_installments_DAYS_ENTRY_PAYMENT_sum_max, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max)


In [72]:
// check number of data lines in dataframe creditRiskdf

val rows = creditRiskdf.select($"TARGET").count()

rows: Long = 183875


In [73]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
//slice a small test list to ensure functionality
//var test_lis = creditRiskFeatures.slice(0,2) //TEST var for codeblock

var highOutlier_list = List[String]() //empty list to append features containing outliers in
var lowOutlier_list = List[String]() //empty list to append features containing outliers in

for (feature <- creditRiskFeatures) {
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
    var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
    //check to see if thresholds are exceeded in the column and count
    var outHigh = creditRiskdf.select(col(feature)).filter(col(feature) > lit(testValHigh)).count()
    var outLow = creditRiskdf.select(col(feature)).filter(col(feature) < lit(testValLow)).count()

    // collate a list of features with outliers > 5% of total data rows
    if (outHigh + outLow >= 0.05*rows ){        
        highOutlier_list = feature :: highOutlier_list
    }
    // collate a list of features with outliers < 1% of total data rows    
    else if (outHigh + outLow > 0 & outHigh + outLow < 0.05*rows){
        lowOutlier_list = feature :: lowOutlier_list
    }
}

highOutlier_list: List[String] = List(WEEKDAY_APPR_PROCESS_START_WEDNESDAY, client_bureau_balance_MONTHS_BALANCE_max_sum, previous_loans_CHANNEL_TYPE_Regional / Local_count, previous_loans_PRODUCT_COMBINATION_Card Street_count_norm, previous_loans_NAME_CONTRACT_STATUS_Refused_count, client_cash_NAME_CONTRACT_STATUS_Signed_count_norm_mean, NAME_HOUSING_TYPE_House / apartment, NAME_FAMILY_STATUS_Separated, WALLSMATERIAL_MODE_Stone, brick, previous_loans_NAME_PRODUCT_TYPE_XNA_count, previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count_norm, client_bureau_balance_STATUS_X_count_sum, previous_loans_CODE_REJECT_REASON_CLIENT_count_norm, previous_loans_CHANNEL_TYPE_Regional / Local_count_norm, previous_loans_NAME_GOODS_CATEGORY_Consumer Electronics_count_norm, client_bureau_balance_STATUS_X...

In [74]:
// drop features in highOutlier_list

val df = creditRiskdf.drop(highOutlier_list:_*)

df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]


In [75]:
val creditRiskdf = df
var clean_df = creditRiskdf

// clean_df.cache()

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]
clean_df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]


In [22]:
// test outlier removal code

// val e1 = lowOutlier_list.slice(1,5)

e1: List[String] = List(OCCUPATION_TYPE_High skill tech staff, client_cash_NAME_CONTRACT_STATUS_Active_count_min, bureau_CNT_CREDIT_PROLONG_mean, previous_loans_NAME_TYPE_SUITE_Other_B_count_norm)


In [76]:
//code block takes features in lowOutlier_list and filters main dataframe against them
//code will generate a subset of main dataset without outliers.

//input feature list conatining features with outliers persisting
for (feature <- lowOutlier_list){
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
    var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
    
    if (testValHigh == 0 & testValLow == 0){}
    else {
        clean_df = clean_df.filter(col(feature) <= lit(testValHigh))filter(
            col(feature) > lit(testValLow))        
    }  
    
}

In [77]:
clean_df.count()

res39: Long = 54836


In [98]:
val creditRiskdf = clean_df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]


In [99]:
// structure of cleansed and reshaped dataset

println("Rows:"+creditRiskdf.count())
println("Columns:"+creditRiskdf.columns.size)

Rows:54836
Columns:134


### Machine learning

In [100]:
// random split of cleansed dataset to training set (70%), testing set (20%)
// and validation set (10%)

var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]


In [101]:
// cache train_df dataset for ML stage
train_df.cache()

res52: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 132 more fields]


### Binomial Logistic Regression

In [102]:
// collate arrays of column names with and without the label "TARGET"

val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET"))

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_HOUR_APPR_PROCESS_START_mean, NAME_FAMILY_STATUS_Married, previous_loans_CNT_PAYMENT_max, AMT_INCOME_TOTAL, previou...

In [103]:
// change datatype of data from String to Float

val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_HOUR_APPR_PROCESS_START_mean, NAME_FAMILY_STATUS_Married, previous_loans_CNT_PAYMENT_max, AMT_INCOME_TOTAL, previou...

In [104]:
// place all features except for "TARGET" under column name "features"

val assembler = new VectorAssembler().setInputCols(
    features).setOutputCol("features")
val df2 = assembler.transform(train_df)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_22fdf12a02bd
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 133 more fields]


In [107]:
df2.schema

res56: org.apache.spark.sql.types.StructType = StructType(StructField(EXT_SOURCE_2,FloatType,true), StructField(EXT_SOURCE_3,FloatType,true), StructField(DAYS_BIRTH,FloatType,true), StructField(AMT_CREDIT,FloatType,true), StructField(AMT_ANNUITY,FloatType,true), StructField(DAYS_ID_PUBLISH,FloatType,true), StructField(previous_loans_CNT_PAYMENT_mean,FloatType,true), StructField(previous_loans_SELLERPLACE_AREA_max,FloatType,true), StructField(DAYS_LAST_PHONE_CHANGE,FloatType,true), StructField(DAYS_REGISTRATION,FloatType,true), StructField(CODE_GENDER_F,FloatType,true), StructField(REGION_POPULATION_RELATIVE,FloatType,true), StructField(client_cash_CNT_INSTALMENT_max_mean,FloatType,true), StructField(bureau_DAYS_CREDIT_mean,FloatType,true), StructField(previous_loans_NAME_CONTRACT_STATUS...

In [109]:
// identify label for logistic regression

val labelIndexer = new StringIndexer().setInputCol(
    "TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_8cc4e6ad6b32
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 134 more fields]


In [119]:
// structure of prepared training data

println("Rows:"+df3.count())
println("Columns:"+df3.columns.size)

Rows:38406
Columns:136


In [120]:
// fit logistic model

val model = new LogisticRegression().fit(df3)

model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_7a00cef86dd0, numClasses = 2, numFeatures = 133


#### Prepare test data

In [122]:
for (colName<-creditRiskFeatures){
     |   test_df=test_df.withColumn(colName,col(colName).cast("Float"))
     | }

val test_df2 = assembler.transform(test_df)
val test_df3 = labelIndexer.fit(test_df2).transform(test_df2)
println("Rows:"+test_df3.count())
println("Columns:"+test_df3.columns.size)

Rows:10960
Columns:136


test_df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 133 more fields]
test_df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 134 more fields]


In [123]:
// review model fitting

val predictions = model.transform(test_df3)

predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 137 more fields]


### Evaluate Model

Determine the metric to evaluate the model, e.g. ROC AUC.

#### Accuracy measurement 1

In [124]:
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.SparkSession


In [125]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println("Test set accuracy = " + accuracy)

Test set accuracy = 0.9317518248175183


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_680437c48553
accuracy: Double = 0.9317518248175183


We can see that the accuracy of our prediction is over 90%. Much higher than our threshold 50%

#### Accuracy measurement 2

Or we use a more intuitive way to check accuracy: We filter the amount of rows in which 'label' and 'prediction' doesn't match (in other word the rows with prediction failure)

In [126]:
predictions.count()

res69: Long = 10960


In [127]:
predictions.filter(!($"label" <=> $"prediction")).select (
    "features", "label", "prediction").show()

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.01810070686042...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.02541386522352...|  1.0|       0.0|
|[0.02625638991594...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.03873163834214...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.04416552931070...|  1.0|       0.0|
|[0.04623770341277...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|[0.05377653241157...|  1.0|       0.0|
|[0.05877300724387...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
|(133,[0,1,2,3,4,5...|  1.0|       0.0|
+--------------------+-----+----------+
only showing top 20 rows



In [128]:
predictions.filter(!($"label" <=> $"prediction")).select (
    "features", "label", "prediction").count()

res71: Long = 748


We can see that among the 10960 rows, the prediction failure is of 748 rows. The accuracy is over 90%

### Model refinement

At this stage, we can apply PCA to select features.  We can also explore algorithm that recursively eliminate features with lowest importance.

Not applicable for binomial logistc regression?!

### Random Forest

In [129]:
// Cross check that the full data set is being used
println("Total data rows in combined dataframe:"+creditRiskdf.count())
var RF_df = creditRiskdf
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   RF_df=RF_df.withColumn(colName,col(colName).cast("Double"))
     | }
//RF_df.printSchema()

Total data rows in combined dataframe:54836


RF_df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 132 more fields]
creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_HOUR_APPR_...

In [130]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}


In [131]:
// Load and parse the data file, converting it to a DataFrame.
val data = RF_df
// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("TARGET")
  .setOutputCol("indexedLabel")
  .fit(data)

data: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 132 more fields]
labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_032e7e45685e


In [132]:
// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val assembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
val RF_df2 = assembler.transform(RF_df)
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(RF_df2)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_3f5638ec4c61
RF_df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 133 more fields]
featureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = vecIdx_efd77a136c58


In [134]:
RF_df2.take(10).foreach(println)

[0.715041819,0.176652579,-13439.0,80865.0,5881.5,-3227.0,17.0,200.0,-2370.0,-311.0,1.0,0.031329,19.34328358,-618.4285714,0.0,1.391891892,-1634.0,0.75,0.326727273,-2339.0,-4329.0,-327.0,13.0,1.0,24.0,67500.0,-3151.0,-39.52238806,-776.9850746,-1369.0,-1369.0,8593.155,14480.46,0.217818182,4.0,18.46268657,0.571428571,-1367.0,1.0,-929.6666667,0.75,0.955223881,-2662.0,1.0,17.46268657,-1779.5,10.0,-2009.0,0.857142857,14.0,-405.8571429,-7118.0,-2648.0,0.081681818,0.0,0.0,-1045.666667,0.0,0.142857143,0.0,0.0,1.0,0.0,1.0,0.0,-2007.0,0.0,1.0,0.25,0.044776119,0.0,3.0,0.0,0.0,55.0,0.0,1.0,0.0,0.0,0.5,0.0,0.333333333,0.0,6.0,0.0,1.0,1.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,-530.0,0.0,0.0,74.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.75,8.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,1.0,0.0,100016.0,(133,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,56,58,61,63,

[0.572064147,0.191821602,-17542.0,1288350.0,37800.0,-988.0,13.6,158.0,-1230.0,-1529.0,1.0,0.003069,15.72,-399.0,0.0,1.266666667,-731.0,0.6,0.435636364,-1199.0,-2394.0,94.0,11.4,1.0,36.0,202500.0,-2599.0,-16.6,-221.14,-644.0,-277.0,12098.367,18734.805,0.435636364,3.0,10.64,0.5,-407.0,1.0,-398.6666667,0.2,0.9,-997.0,1.0,9.48,-784.6,11.0,-989.0,0.666666667,14.0,-174.0,-3923.0,-830.0,0.145212121,0.0,0.0,-514.2,0.0,0.333333333,0.0,0.4,0.6,0.0,1.0,0.0,-984.0,0.0,1.0,0.4,0.1,0.0,3.0,0.4,0.2,50.0,0.0,0.6,0.0,0.4,0.2,0.2,0.4,0.0,3.5,0.0,0.8,2.0,0.0,0.0,0.0,12.0,0.0,0.4,0.0,6.0,0.2,3.0,0.0,-40.0,0.2,0.0,45.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.923076923,6.0,0.0,0.0,1.0,0.0,2.0,0.0,0.2,6.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,100087.0,[0.572064147,0.191821602,-17542.0,1288350.0,37800.0,-988.0,13.6,158.0,-1230.0,-1529.0,1.0,0.003069,15.72,-399.0,0.0,1.266666667,-731.0,0.6,0.435636364,-1199.0,-2394.0,94.0,11.4,1.0,36.0,202500.0,-2599.0,-16.6,-221.14,-644.0,-277.0,12098.367,18734.805,0.

In [135]:
// Split the data into training, test and validation sets (20% held out for testing).
val Array(trainingData, testData, validationData) = RF_df2.randomSplit(
    Array(0.7, 0.2,0.1))

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 133 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 133 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 133 more fields]


In [136]:
// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(20)

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_82214be1176c


In [137]:
// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_0362d1d7f46b


In [138]:
// Chain indexers and forest in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))

pipeline: org.apache.spark.ml.Pipeline = pipeline_e302510a083b


In [139]:
// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

model: org.apache.spark.ml.PipelineModel = pipeline_e302510a083b


In [140]:
// Make predictions.
val predictions = model.transform(testData)

predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: double, EXT_SOURCE_3: double ... 139 more fields]


In [ ]:
// Select example rows to display.

// predictions.select("predictedLabel", "probability", "features").show(100)

In [141]:
// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.06406179009541113


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5a77dae2654d
accuracy: Double = 0.9359382099045889


We can see that the accuracy of our prediction is over 90%. Much higher than our threshold 50%

In [142]:
val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
println(s"Learned classification forest model:\n ${rfModel.toDebugString}")

Learned classification forest model:
 RandomForestClassificationModel (uid=rfc_82214be1176c) with 20 trees
  Tree 0 (weight 1.0):
    If (feature 29 <= -902.5)
     If (feature 24 <= 33.0)
      If (feature 17 <= 0.39230769249999997)
       Predict: 0.0
      Else (feature 17 > 0.39230769249999997)
       If (feature 33 <= 0.11669819349999999)
        Predict: 0.0
       Else (feature 33 > 0.11669819349999999)
        If (feature 124 <= 0.4166666665)
         Predict: 0.0
        Else (feature 124 > 0.4166666665)
         Predict: 1.0
     Else (feature 24 > 33.0)
      If (feature 36 <= 0.3431372545)
       Predict: 0.0
      Else (feature 36 > 0.3431372545)
       If (feature 1 <= 0.4516404205)
        Predict: 0.0
       Else (feature 1 > 0.4516404205)
        If (feature 82 <= 0.6201923075)
         Predict: 0.0
        Else (feature 82 > 0.6201923075)
         Predict: 1.0
    Else (feature 29 > -902.5)
     If (feature 34 <= 4.5)
      Predict: 0.0
     Else (feature 34 > 4.5)
  

         Predict: 1.0
      Else (feature 1 > 0.4713547295)
       Predict: 0.0
  Tree 12 (weight 1.0):
    If (feature 18 <= 0.10425844000000001)
     If (feature 39 <= -1051.8333335)
      If (feature 43 <= 0.44097222199999997)
       If (feature 1 <= 0.239883848)
        If (feature 111 <= 8.5)
         Predict: 1.0
        Else (feature 111 > 8.5)
         Predict: 0.0
       Else (feature 1 > 0.239883848)
        If (feature 17 <= 0.6125)
         Predict: 0.0
        Else (feature 17 > 0.6125)
         Predict: 1.0
      Else (feature 43 > 0.44097222199999997)
       Predict: 0.0
     Else (feature 39 > -1051.8333335)
      If (feature 91 <= 0.5)
       Predict: 0.0
      Else (feature 91 > 0.5)
       If (feature 45 <= -1566.325)
        If (feature 82 <= 0.480769231)
         Predict: 0.0
        Else (feature 82 > 0.480769231)
         Predict: 1.0
       Else (feature 45 > -1566.325)
        If (feature 70 <= 58.5)
         Predict: 0.0
        Else (feature 70 > 58.5)
      

         Predict: 0.0
      Else (feature 64 > 0.345238095)
       If (feature 36 <= 0.218253968)
        If (feature 74 <= 2.5)
         Predict: 1.0
        Else (feature 74 > 2.5)
         Predict: 0.0
       Else (feature 36 > 0.218253968)
        If (feature 19 <= -2732.5)
         Predict: 0.0
        Else (feature 19 > -2732.5)
         Predict: 1.0
     Else (feature 29 > -664.0)
      If (feature 5 <= -1588.5)
       If (feature 10 in {1.0})
        Predict: 0.0
       Else (feature 10 not in {1.0})
        If (feature 64 <= 0.4083333335)
         Predict: 0.0
        Else (feature 64 > 0.4083333335)
         Predict: 1.0
      Else (feature 5 > -1588.5)
       Predict: 0.0
    Else (feature 24 > 29.5)
     If (feature 52 <= -803.5)
      If (feature 13 <= -498.83333335)
       Predict: 0.0
      Else (feature 13 > -498.83333335)
       If (feature 25 <= 62775.0)
        If (feature 12 <= 12.56971154)
         Predict: 1.0
        Else (feature 12 > 12.56971154)
         Predi

rfModel: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_82214be1176c) with 20 trees
